In [2]:
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets
from threading import Thread
import time

class PalmDetectionWarning:
    def __init__(self):
        # Initialize MediaPipe
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )
        self.mp_draw = mp.solutions.drawing_utils
        
        # Warning system
        self.palm_detected = False
        self.warning_duration = 0
        self.last_detection_time = 0
        
    def is_palm_open(self, landmarks):
        """
        Detect if palm is open by checking finger positions
        Returns True if palm is open (all fingers extended)
        """
        # Finger tip and pip landmarks
        finger_tips = [4, 8, 12, 16, 20]  # Thumb, Index, Middle, Ring, Pinky tips
        finger_pips = [3, 6, 10, 14, 18]  # Corresponding PIP joints
        
        fingers_up = 0
        
        # Check thumb (different logic due to thumb orientation)
        if landmarks[finger_tips[0]].x > landmarks[finger_pips[0]].x:
            fingers_up += 1
            
        # Check other four fingers
        for i in range(1, 5):
            if landmarks[finger_tips[i]].y < landmarks[finger_pips[i]].y:
                fingers_up += 1
                
        # Palm is open if 4 or 5 fingers are up
        return fingers_up >= 4
    
    def draw_warning(self, frame):
        """Draw warning overlay on frame"""
        height, width = frame.shape[:2]
        
        # Create warning overlay
        overlay = frame.copy()
        
        # Red warning background
        cv2.rectangle(overlay, (0, 0), (width, 100), (0, 0, 255), -1)
        
        # Warning text
        warning_text = "⚠️ PALM DETECTED - WARNING! ⚠️"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.2
        thickness = 3
        
        # Get text size for centering
        text_size = cv2.getTextSize(warning_text, font, font_scale, thickness)[0]
        text_x = (width - text_size[0]) // 2
        text_y = 60
        
        # Draw warning text
        cv2.putText(overlay, warning_text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness)
        
        # Blend overlay with original frame
        alpha = 0.8
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
        
        return frame
    
    def process_frame(self, frame):
        """Process single frame for palm detection"""
        # Convert BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        
        palm_detected_in_frame = False
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks
                self.mp_draw.draw_landmarks(
                    frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
                
                # Check if palm is open
                if self.is_palm_open(hand_landmarks.landmark):
                    palm_detected_in_frame = True
                    
                    # Draw bounding box around detected palm
                    h, w, c = frame.shape
                    cx_min = int(min([lm.x for lm in hand_landmarks.landmark]) * w)
                    cy_min = int(min([lm.y for lm in hand_landmarks.landmark]) * h)
                    cx_max = int(max([lm.x for lm in hand_landmarks.landmark]) * w)
                    cy_max = int(max([lm.y for lm in hand_landmarks.landmark]) * h)
                    
                    # Draw green bounding box
                    cv2.rectangle(frame, (cx_min, cy_min), (cx_max, cy_max), (0, 255, 0), 2)
                    cv2.putText(frame, 'PALM DETECTED', (cx_min, cy_min-10), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Update palm detection status
        if palm_detected_in_frame:
            self.palm_detected = True
            self.last_detection_time = time.time()
            self.warning_duration = 2.0  # Show warning for 2 seconds
        
        # Check if we should show warning
        current_time = time.time()
        if current_time - self.last_detection_time < self.warning_duration:
            frame = self.draw_warning(frame)
        else:
            self.palm_detected = False
            
        return frame
    
    def start_detection(self):
        """Start palm detection from webcam"""
        cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("Error: Could not open webcam")
            return
            
        print("Palm Detection Started!")
        print("Press 'q' to quit")
        print("Show your palm(s) to the camera to trigger warning")
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("Error: Could not read frame")
                    break
                
                # Flip frame horizontally for mirror effect
                frame = cv2.flip(frame, 1)
                
                # Process frame
                processed_frame = self.process_frame(frame)
                
                # Display frame
                cv2.imshow('Palm Detection Warning System', processed_frame)
                
                # Exit on 'q' key press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        except KeyboardInterrupt:
            print("Detection stopped by user")
        finally:
            cap.release()
            cv2.destroyAllWindows()
            print("Palm detection stopped")

# Create and run the palm detection system
detector = PalmDetectionWarning()


In [3]:
# Start the palm detection system
detector.start_detection()


Palm Detection Started!
Press 'q' to quit
Show your palm(s) to the camera to trigger warning
Palm detection stopped


In [1]:
import cv2
import mediapipe as mp
import numpy as np
import threading
import queue
import time

class VideoStream:
    """Your optimized video streaming class for IP cameras"""
    def __init__(self, src):
        self.cap = cv2.VideoCapture(src)
        # Optimize capture settings
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        self.cap.set(cv2.CAP_PROP_FPS, 30) # Set desired FPS
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640) # Reduce resolution
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480) # Reduce resolution
        
        self.q = queue.Queue(maxsize=2) # Small buffer
        self.running = True
        
    def start(self):
        self.thread = threading.Thread(target=self.update)
        self.thread.start()
        return self
        
    def update(self):
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                break
            
            # Clear queue if it's full (drop old frames)
            if not self.q.empty():
                try:
                    self.q.get_nowait()
                except queue.Empty:
                    pass
            
            self.q.put(frame)
            
    def read(self):
        try:
            return self.q.get(timeout=1.0)
        except queue.Empty:
            return None
            
    def stop(self):
        self.running = False
        self.thread.join()
        self.cap.release()

class IPPalmDetectionWarning:
    """MediaPipe Palm Detection with IP Camera Support and Warning System"""
    
    def __init__(self, ip_address):
        # Initialize MediaPipe
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )
        self.mp_draw = mp.solutions.drawing_utils
        
        # IP Camera setup
        self.ip_address = ip_address
        self.video_stream = None
        
        # Warning system
        self.palm_detected = False
        self.warning_duration = 2.0  # Show warning for 2 seconds
        self.last_detection_time = 0
        self.detection_count = 0
        
        # Performance monitoring
        self.fps_start_time = time.time()
        self.fps_frame_count = 0
        
        # Frame processing optimization
        self.frame_skip = 2  # Process every 2nd frame for better performance
        self.frame_count = 0
        self.last_processed_frame = None
        
    def is_palm_open(self, landmarks):
        """
        Detect if palm is open by checking finger positions
        Returns True if palm is open (all fingers extended)
        """
        # Finger tip and pip landmarks
        finger_tips = [4, 8, 12, 16, 20]  # Thumb, Index, Middle, Ring, Pinky tips
        finger_pips = [3, 6, 10, 14, 18]  # Corresponding PIP joints
        
        fingers_up = 0
        
        # Check thumb (different logic due to thumb orientation)
        if landmarks[finger_tips[0]].x > landmarks[finger_pips[0]].x:
            fingers_up += 1
            
        # Check other four fingers
        for i in range(1, 5):
            if landmarks[finger_tips[i]].y < landmarks[finger_pips[i]].y:
                fingers_up += 1
                
        # Palm is open if 4 or 5 fingers are up
        return fingers_up >= 4
    
    def draw_warning_overlay(self, frame):
        """Draw prominent warning overlay when palm is detected"""
        height, width = frame.shape[:2]
        
        # Create warning overlay
        overlay = frame.copy()
        
        # Flashing red warning background
        flash_intensity = int(abs(np.sin(time.time() * 8)) * 255)  # Flashing effect
        warning_color = (0, 0, flash_intensity)
        
        # Main warning banner
        cv2.rectangle(overlay, (0, 0), (width, 120), warning_color, -1)
        
        # Warning text
        warning_text = "⚠️ PALM DETECTED - WARNING! ⚠️"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1.5
        thickness = 3
        
        # Get text size for centering
        text_size = cv2.getTextSize(warning_text, font, font_scale, thickness)[0]
        text_x = (width - text_size[0]) // 2
        text_y = 70
        
        # Draw warning text with outline
        cv2.putText(overlay, warning_text, (text_x, text_y), font, font_scale, (0, 0, 0), thickness + 2)  # Outline
        cv2.putText(overlay, warning_text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness)  # Text
        
        # Detection counter
        counter_text = f"Detection Count: {self.detection_count}"
        cv2.putText(overlay, counter_text, (20, 100), font, 0.8, (255, 255, 255), 2)
        
        # Blend overlay with original frame
        alpha = 0.85
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
        
        return frame
    
    def draw_status_info(self, frame):
        """Draw status information on frame"""
        height, width = frame.shape[:2]
        
        # Status box
        status_box_height = 60
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, height - status_box_height), (width, height), (0, 0, 0), -1)
        frame = cv2.addWeighted(overlay, 0.7, frame, 0.3, 0)
        
        # Status text
        status_text = f"IP Stream: {self.ip_address.split('/')[-2] if '/' in self.ip_address else 'Connected'}"
        cv2.putText(frame, status_text, (10, height - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # Palm detection status
        if self.palm_detected:
            detection_status = "PALM ACTIVE"
            color = (0, 0, 255)  # Red
        else:
            detection_status = "MONITORING"
            color = (0, 255, 0)  # Green
            
        cv2.putText(frame, detection_status, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # FPS display
        cv2.putText(frame, f"FPS: {self.get_current_fps():.1f}", (width - 100, height - 35), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        return frame
    
    def get_current_fps(self):
        """Calculate current FPS"""
        current_time = time.time()
        if hasattr(self, 'last_fps_time'):
            fps = 1.0 / (current_time - self.last_fps_time) if (current_time - self.last_fps_time) > 0 else 0
        else:
            fps = 0
        self.last_fps_time = current_time
        return fps
    
    def process_frame(self, frame):
        """Process single frame for palm detection"""
        # Convert BGR to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        
        palm_detected_in_frame = False
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks
                self.mp_draw.draw_landmarks(
                    frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS,
                    self.mp_draw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    self.mp_draw.DrawingSpec(color=(255, 0, 0), thickness=2)
                )
                
                # Check if palm is open
                if self.is_palm_open(hand_landmarks.landmark):
                    palm_detected_in_frame = True
                    self.detection_count += 1
                    
                    # Draw bounding box around detected palm
                    h, w, c = frame.shape
                    cx_min = int(min([lm.x for lm in hand_landmarks.landmark]) * w)
                    cy_min = int(min([lm.y for lm in hand_landmarks.landmark]) * h)
                    cx_max = int(max([lm.x for lm in hand_landmarks.landmark]) * w)
                    cy_max = int(max([lm.y for lm in hand_landmarks.landmark]) * h)
                    
                    # Draw bright green bounding box
                    cv2.rectangle(frame, (cx_min-10, cy_min-10), (cx_max+10, cy_max+10), (0, 255, 0), 3)
                    cv2.putText(frame, 'PALM OPEN!', (cx_min, cy_min-20), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        # Update palm detection status
        if palm_detected_in_frame:
            self.palm_detected = True
            self.last_detection_time = time.time()
        
        # Check if we should show warning (show for warning_duration seconds)
        current_time = time.time()
        if current_time - self.last_detection_time < self.warning_duration:
            frame = self.draw_warning_overlay(frame)
        else:
            self.palm_detected = False
        
        # Always draw status info
        frame = self.draw_status_info(frame)
            
        return frame
    
    def start_detection(self):
        """Start palm detection from IP camera"""
        print(f"🎥 Connecting to IP camera: {self.ip_address}")
        
        # Initialize video stream
        self.video_stream = VideoStream(self.ip_address).start()
        
        # Wait a moment for stream to initialize
        time.sleep(2.0)
        
        print("🚀 Palm Detection Started!")
        print("📺 IP Camera stream active")
        print("✋ Show your palm(s) to trigger warnings")
        print("❌ Press 'q' to quit")
        print("-" * 60)
        
        try:
            while True:
                frame = self.video_stream.read()
                if frame is None:
                    print('⚠️ No frame received from IP camera')
                    continue
                
                self.frame_count += 1
                
                # Process every nth frame for better performance
                if self.frame_count % self.frame_skip == 0:
                    self.last_processed_frame = self.process_frame(frame.copy())
                
                # Always display the last processed frame or current frame
                display_frame = self.last_processed_frame if self.last_processed_frame is not None else frame
                
                # Add frame counter for debugging
                cv2.putText(display_frame, f"Frame: {self.frame_count}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                # Display frame
                cv2.imshow('IP Camera Palm Detection Warning System', display_frame)
                
                # FPS monitoring (print every 60 frames)
                self.fps_frame_count += 1
                if self.fps_frame_count % 60 == 0:
                    fps_end_time = time.time()
                    fps = 60 / (fps_end_time - self.fps_start_time)
                    print(f"📊 FPS: {fps:.1f} | Detections: {self.detection_count}")
                    self.fps_start_time = time.time()
                
                # Exit on 'q' key press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    print('\n🛑 Detection stopped by user')
                    break
                    
        except KeyboardInterrupt:
            print('\n🛑 Detection interrupted by user')
        except Exception as e:
            print(f'❌ Error: {e}')
        finally:
            if self.video_stream:
                self.video_stream.stop()
            cv2.destroyAllWindows()
            print('✅ System shutdown complete')

# Usage Example
if __name__ == "__main__":
    # Configure your IP camera address
    IP_ADDRESS = 'http://192.168.0.111:4747/video'  # Replace with your IP camera URL
    
    # Create and start the detection system
    detector = IPPalmDetectionWarning(IP_ADDRESS)
    detector.start_detection()


🎥 Connecting to IP camera: http://192.168.0.111:4747/video
🚀 Palm Detection Started!
📺 IP Camera stream active
✋ Show your palm(s) to trigger warnings
❌ Press 'q' to quit
------------------------------------------------------------
📊 FPS: 8.0 | Detections: 0
📊 FPS: 21.9 | Detections: 3
📊 FPS: 17.5 | Detections: 6
📊 FPS: 24.5 | Detections: 13
⚠️ No frame received from IP camera
📊 FPS: 13.4 | Detections: 14
📊 FPS: 20.6 | Detections: 14
📊 FPS: 23.4 | Detections: 14
📊 FPS: 24.0 | Detections: 14
📊 FPS: 19.5 | Detections: 14
📊 FPS: 19.4 | Detections: 14
📊 FPS: 20.0 | Detections: 24
📊 FPS: 19.2 | Detections: 26
📊 FPS: 24.6 | Detections: 26
📊 FPS: 23.1 | Detections: 26
📊 FPS: 24.9 | Detections: 26
📊 FPS: 23.8 | Detections: 26
📊 FPS: 19.9 | Detections: 43
📊 FPS: 20.3 | Detections: 48
📊 FPS: 17.5 | Detections: 63
📊 FPS: 25.3 | Detections: 73
📊 FPS: 24.9 | Detections: 73

🛑 Detection stopped by user
✅ System shutdown complete
